In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, Concatenate

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
# Custom function to build hidden combination (i.e. convolution) layers

def create_random_combination_layer(input_layer, combination_size, num_combinations, input_dim):
    outputs = []
    
    for _ in range(num_combinations):
        # First random feature selection
        indices_1 = np.random.choice(input_dim, combination_size, replace=False)
        indices_tensor_1 = tf.constant(indices_1, dtype=tf.int32)
        
        # First feature selection using Lambda layer
        slice_layer_1 = Lambda(
            lambda x: tf.gather(x, indices_tensor_1, axis=1),  # Gather selected features
            output_shape=(combination_size,)
        )(input_layer)
        
        # Second random feature selection (after the first random selection)
        indices_2 = np.random.choice(combination_size, combination_size, replace=False)
        indices_tensor_2 = tf.constant(indices_2, dtype=tf.int32)
        
        # Second feature selection using Lambda layer
        slice_layer_2 = Lambda(
            lambda x: tf.gather(x, indices_tensor_2, axis=1),  # Apply a second feature selection
            output_shape=(combination_size,)
        )(slice_layer_1)

        # Apply Dense layers on the final selected subset
        selected_features = Dense(16, activation='relu')(
            Dense(8, activation='relu')(
                Dense(4, activation='relu')(slice_layer_2)
            )
        )
        outputs.append(selected_features)
    
    # Concatenate the outputs from all the random feature combinations
    return Concatenate()(outputs)

In [ ]:
def weighted_categorical_crossentropy(weights):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, 'float32')
        weights_tensor = K.constant(weights)
        weights_applied = y_true * weights_tensor
        return K.categorical_crossentropy(y_true, y_pred) * K.sum(weights_applied, axis=-1)
    return loss


In [ ]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Perform a stratified split into train and validation sets (80% train, 20% validation)
    X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42, stratify=train_y)
    del train_X
    del train_y
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # Parameters for Random Feature Combination
    num_combinations = 20  # Number of random column combinations
    combination_size = 3   # Number of columns in each combination

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 5

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):

        # Compute class weights for training data
        class_weights = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train.argmax(axis=1)),  # Convert one-hot back to class indices
            y=y_train.argmax(axis=1)
        )

        # Convert class weights to a dictionary
        class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        input_layer = Input(shape=(X_train.shape[1],))  # Input shape from the training data
        feature_layer = create_random_combination_layer(input_layer, combination_size, num_combinations, X_train.shape[1])
        hidden_layer = Dense(128, activation='relu')(feature_layer)
        hidden_layer = Dropout(0.5)(hidden_layer)
        output_layer = Dense(test_y.shape[1], activation='softmax')(hidden_layer)
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(optimizer='adam', loss=weighted_categorical_crossentropy(class_weights_dict), metrics=['accuracy'])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            X_train, y_train, 
            epochs=1000, 
            batch_size=int(X_train.shape[0] * 0.01), 
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            class_weight=class_weights_dict,
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

    # Average the metrics over all runs
    print("\nAggregated Metrics:")
    for metric, values in metrics_storage.items():
        avg_value = np.mean(values)
        print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [7]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 0.4754
test_accuracy: 0.8232
balanced_accuracy: 0.8140
roc_auc: 0.9397
pr_auc: 0.9488
f2: 0.8388
mcc: 0.7159
0_precision: 0.9892
0_recall: 0.8137
0_f1-score: 0.8817
0_support: 9117.0000
1_precision: 0.5310
1_recall: 0.9600
1_f1-score: 0.5696
1_support: 10.0000
2_precision: 0.3606
2_recall: 0.7118
2_f1-score: 0.4047
2_support: 34.0000
3_precision: 0.7634
3_recall: 0.8259
3_f1-score: 0.7919
3_support: 1781.0000
4_precision: 0.9602
4_recall: 0.9531
4_f1-score: 0.9566
4_support: 653.0000
5_precision: 0.4221
5_recall: 0.7000
5_f1-score: 0.4374
5_support: 2.0000
6_precision: 0.1736
6_recall: 0.7333
6_f1-score: 0.2443
6_support: 3.0000
accuracy: 0.8232
macro avg_precision: 0.6000
macro avg_recall: 0.8140
macro avg_f1-score: 0.6123
macro avg_support: 11600.0000
weighted avg_precision: 0.9504
weighted avg_recall: 0.8232
weighted avg_f1-score: 0.8702
weighted avg_support

In [8]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.8358
test_accuracy: 0.1715
balanced_accuracy: 0.2315
roc_auc: 0.6013
pr_auc: 0.4116
f2: 0.1663
mcc: 0.0576
0_precision: 0.2689
0_recall: 0.0841
0_f1-score: 0.1078
0_support: 42368.0000
1_precision: 0.4636
1_recall: 0.2173
1_f1-score: 0.2832
1_support: 56661.0000
2_precision: 0.1579
2_recall: 0.2653
2_f1-score: 0.0973
2_support: 7151.0000
3_precision: 0.0177
3_recall: 0.4619
3_f1-score: 0.0331
3_support: 549.0000
4_precision: 0.0523
4_recall: 0.1724
4_f1-score: 0.0620
4_support: 1899.0000
5_precision: 0.0344
5_recall: 0.2318
5_f1-score: 0.0598
5_support: 3473.0000
6_precision: 0.1174
6_recall: 0.1878
6_f1-score: 0.1009
6_support: 4102.0000
accuracy: 0.1715
macro avg_precision: 0.1589
macro avg_recall: 0.2315
macro avg_f1-score: 0.1063
macro avg_support: 116203.0000
weighted avg_precision: 0.3399
weighted avg_recall: 0.1715
weighted avg_f1-score: 0.1899
weighte

In [9]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("/kaggle/input/ma-datasets/kdd_train.csv", "/kaggle/input/ma-datasets/kdd_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.9731
test_accuracy: 0.0948
balanced_accuracy: 0.1965
roc_auc: 0.6988
pr_auc: 0.9266
f2: 0.1025
mcc: 0.0720
0_precision: 0.5986
0_recall: 0.0834
0_f1-score: 0.1362
0_support: 194557.0000
1_precision: 0.2859
1_recall: 0.5393
1_f1-score: 0.3162
1_support: 3178.0000
2_precision: 0.0000
2_recall: 0.0000
2_f1-score: 0.0000
2_support: 2496.0000
3_precision: 0.0588
3_recall: 0.3534
3_f1-score: 0.1006
3_support: 2083.0000
4_precision: 0.0024
4_recall: 0.2285
4_f1-score: 0.0048
4_support: 463.0000
5_precision: 0.3111
5_recall: 0.8649
5_f1-score: 0.3503
5_support: 441.0000
6_precision: 0.0415
6_recall: 0.1451
6_f1-score: 0.0432
6_support: 204.0000
7_precision: 0.0044
7_recall: 0.4673
7_f1-score: 0.0088
7_support: 196.0000
8_precision: 0.0014
8_recall: 0.4302
8_f1-score: 0.0028
8_support: 53.0000
9_precision: 0.0070
9_recall: 0.1818
9_f1-score: 0.0134
9_support: 11.0000


In [10]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/5 started...
Run 2/5 started...
Run 3/5 started...
Run 4/5 started...
Run 5/5 started...

Aggregated Metrics:
test_loss: 1.8968
test_accuracy: 0.2827
balanced_accuracy: 0.3864
roc_auc: 0.7023
pr_auc: 0.7834
f2: 0.2725
mcc: 0.1380
0_precision: 0.7604
0_recall: 0.2810
0_f1-score: 0.3156
0_support: 26862.0000
1_precision: 0.2076
1_recall: 0.1387
1_f1-score: 0.1652
1_support: 2657.0000
2_precision: 0.2351
2_recall: 0.6771
2_f1-score: 0.2754
2_support: 908.0000
3_precision: 0.0504
3_recall: 0.3107
3_f1-score: 0.0567
3_support: 522.0000
4_precision: 0.0413
4_recall: 0.3836
4_f1-score: 0.0740
4_support: 293.0000
5_precision: 0.0261
5_recall: 0.3033
5_f1-score: 0.0471
5_support: 269.0000
6_precision: 0.0040
6_recall: 0.2190
6_f1-score: 0.0078
6_support: 116.0000
7_precision: 0.0189
7_recall: 0.5321
7_f1-score: 0.0363
7_support: 53.0000
8_precision: 0.0883
8_recall: 0.6318
8_f1-score: 0.1513
8_support: 44.0000
accuracy: 0.2827
macro avg_precision: 0.1591
macro avg_recall: 0.3864
macro avg